In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'Property_with_Feature_Engineering.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

# Display the first 5 rows of the data
df.columns

Index(['property_id', 'location_id', 'page_url', 'property_type', 'price',
       'price_bin', 'location', 'city', 'province_name', 'locality',
       'latitude', 'longitude', 'baths', 'area', 'area_marla', 'area_sqft',
       'purpose', 'bedrooms', 'date_added', 'year', 'month', 'day', 'agency',
       'agent'],
      dtype='object')

In [4]:
# Create the Property DataFrame
property_columns = ['property_id', 'property_type', 'price', 'price_bin', 'area', 
                    'area_marla', 'area_sqft', 'baths', 'bedrooms', 'purpose', 
                    'year', 'month', 'day', 'location_id', 'agent', 'agency']
property_df = df[property_columns].copy()

# Create a date_added column by combining year, month, and day
property_df['date_added'] = pd.to_datetime(property_df[['year', 'month', 'day']])
property_df.drop(columns=['year', 'month', 'day'], inplace=True)

# Create the Location DataFrame
location_columns = ['location_id', 'city', 'province_name', 'locality', 'latitude', 'longitude']
location_df = df[location_columns].drop_duplicates().copy()

# Create the Agency DataFrame
agency_df = df[['agency']].drop_duplicates().reset_index(drop=True).copy()
agency_df['agency_id'] = agency_df.index + 1  # Create agency_id as a new primary key

# Merge agency information into property_df
property_df = pd.merge(property_df, agency_df, on='agency', how='left')

# Create the Agent DataFrame
agent_df = df[['agent']].drop_duplicates().reset_index(drop=True).copy()
agent_df['agent_id'] = agent_df.index + 1  # Create agent_id as a new primary key

# Merge agent information into property_df
property_df = pd.merge(property_df, agent_df, on='agent', how='left')

# Drop agency and agent names from property_df since we have their IDs now
property_df.drop(columns=['agency', 'agent'], inplace=True)

# Display the DataFrames for each entity
print("Property DataFrame:")
print(property_df.columns)

print("\nLocation DataFrame:")
print(location_df.columns)

print("\nAgency DataFrame:")
print(agency_df.columns)

print("\nAgent DataFrame:")
print(agent_df.columns)

Property DataFrame:
Index(['property_id', 'property_type', 'price', 'price_bin', 'area',
       'area_marla', 'area_sqft', 'baths', 'bedrooms', 'purpose',
       'location_id', 'date_added', 'agency_id', 'agent_id'],
      dtype='object')

Location DataFrame:
Index(['location_id', 'city', 'province_name', 'locality', 'latitude',
       'longitude'],
      dtype='object')

Agency DataFrame:
Index(['agency', 'agency_id'], dtype='object')

Agent DataFrame:
Index(['agent', 'agent_id'], dtype='object')


In [7]:
import pandas as pd
from sqlalchemy import create_engine, text

# PostgreSQL cloud database credentials
db_config = {
    'user': 'postgres',  # Your PostgreSQL username
    'password': 'BestPasswordEver',  # Your PostgreSQL password
    'host': '34.133.41.198',  # Replace with your VM/Cloud Instance IP or hostname
    'port': '5432',  # PostgreSQL port
    'database': 'postgres'  # Database name
}

# Create SQLAlchemy engine for PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Function to create the tables with relationships
def create_tables():
    with engine.connect() as conn:
        # Create Location Table
        conn.execute(text("""
        CREATE TABLE IF NOT EXISTS location (
            location_id SERIAL PRIMARY KEY,
            city VARCHAR(100),
            province_name VARCHAR(100),
            locality VARCHAR(255),
            latitude DECIMAL,
            longitude DECIMAL
        );
        """))

        # Create Agency Table
        conn.execute(text("""
        CREATE TABLE IF NOT EXISTS agency (
            agency_id SERIAL PRIMARY KEY,
            agency_name VARCHAR(255)
        );
        """))

        # Create Agent Table
        conn.execute(text("""
        CREATE TABLE IF NOT EXISTS agent (
            agent_id SERIAL PRIMARY KEY,
            agent_name VARCHAR(255)
        );
        """))

        # Create Property Table with foreign keys
        conn.execute(text("""
        CREATE TABLE IF NOT EXISTS property (
            property_id SERIAL PRIMARY KEY,
            property_type VARCHAR(100),
            price DECIMAL,
            price_bin VARCHAR(50),
            area DECIMAL,
            area_marla DECIMAL,
            area_sqft DECIMAL,
            baths INT,
            bedrooms INT,
            purpose VARCHAR(50),
            date_added DATE,
            location_id INT,
            agency_id INT,
            agent_id INT,
            FOREIGN KEY (location_id) REFERENCES location(location_id),
            FOREIGN KEY (agency_id) REFERENCES agency(agency_id),
            FOREIGN KEY (agent_id) REFERENCES agent(agent_id)
        );
        """))

        print("Tables created successfully")

# Function to upload the data
def upload_data():
    # Upload data into the tables
    # 1. Insert Location Data
    location_df.to_sql('location', engine, if_exists='append', index=False)
    
    # 2. Insert Agency Data
    agency_df.to_sql('agency', engine, if_exists='append', index=False)
    
    # 3. Insert Agent Data
    agent_df.to_sql('agent', engine, if_exists='append', index=False)
    
    # 4. Insert Property Data
    property_df.to_sql('property', engine, if_exists='append', index=False)
    
    print("Data uploaded successfully")

# Call functions to create tables and upload data
create_tables()
upload_data()

Tables created successfully
Data uploaded successfully


In [11]:
import pandas as pd
from sqlalchemy import create_engine

# Your database credentials
DATABASE_TYPE = 'postgresql'
DB_USER = 'postgres'  # Your PostgreSQL username
DB_PASSWORD = 'BestPasswordEver'  # Your PostgreSQL password
DB_HOST = '34.133.41.198'  # External IP of your VM or database host
DB_PORT = '5432'  # Default PostgreSQL port
DB_NAME = 'postgres'  # Database name

# Create an SQLAlchemy engine
engine = create_engine(f'{DATABASE_TYPE}://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# Connect to the database and verify the connection
try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL")
        
        # Fetch data from a specific table
        table_name = 'Property'  # Change this to your actual table name
        query = f"SELECT * FROM {table_name}"
        
        # Load data into a DataFrame
        data = pd.read_sql(query, connection)

        # Display the first few rows of the DataFrame to verify
        print(data.head())

except Exception as e:
    print(f"An error occurred: {e}")


Connected to PostgreSQL
   property_id property_type      price  price_bin      area  area_marla  \
0       347795         House  220000000  Very High   6 Kanal       120.0   
1       482892         House   40000000  Very High   1 Kanal        20.0   
2       555962         House    9500000        Low   9 Marla         9.0   
3       562843         House  125000000  Very High   1 Kanal        20.0   
4       686990         House   21000000       High  11 Marla        11.0   

   area_sqft  baths  bedrooms   purpose  location_id date_added  agency_id  \
0   32670.12      0         0  For Sale            8 2019-07-17          1   
1    5445.02      5         5  For Sale           48 2018-10-06          2   
2    2450.26      0         3  For Sale           75 2019-07-03          3   
3    5445.02      7         8  For Sale         3821 2019-04-04          4   
4    2994.76      5         6  For Sale         3522 2019-04-04          4   

   agent_id  
0         1  
1         2  
2       